**🔧 Setup Required**: Before running this notebook, please follow the [setup instructions](../../README.md#setup-instructions) to configure your environment and API keys.

# Multimodal Indexing Pipeline with LLM Content Extraction (Approach 2)

This notebook demonstrates how to build a multimodal indexing pipeline using LLM-based content extraction.

## Approach 2: LLM Content Extraction

In this approach, we use a vision LLM to extract textual descriptions from images, then embed everything as text.

**Pipeline Flow**:
1. Images → Vision LLM → Text descriptions
2. PDFs → Text extraction
3. All text → Text-only embeddings

### Advantages:
- Richer semantic understanding
- More accurate retrieval for conceptual searches
- Better for text-based queries

### Trade-offs:
- Slower (requires LLM calls)
- More expensive (API costs)
- Loses some visual information

### Use Cases:
- Question answering about images
- Document understanding
- Semantic search across modalities

## Setup

In [1]:
import os

notebook_dir = os.path.dirname(os.path.abspath("3_multimodal_indexing_llm.ipynb")) if os.path.exists("3_multimodal_indexing_llm.ipynb") else os.getcwd()

## Download Sample Data

In [2]:
import urllib.request
import ssl

# Create SSL context
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

def download_file(url, filename):
    """Download file with SSL context"""
    req = urllib.request.Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    with urllib.request.urlopen(req, context=ssl_context) as response:
        with open(filename, 'wb') as out_file:
            out_file.write(response.read())

# Create directories
os.makedirs("images", exist_ok=True)
os.makedirs("data_for_indexing", exist_ok=True)

# Download images
download_file(
    "https://upload.wikimedia.org/wikipedia/commons/2/26/Pink_Lady_Apple_%284107712628%29.jpg",
    "images/apple.jpg"
)

# Download PDF
download_file(
    "https://arxiv.org/pdf/1706.03762",
    "data_for_indexing/attention_is_all_you_need.pdf"
)

print("Data downloaded successfully!")

Data downloaded successfully!


## Initialize Pipeline Components

We'll use GPT-4o-mini as the vision LLM and a high-quality text embedder.

**Key Components**:
- `LLMDocumentContentExtractor`: Extracts text descriptions from images using a vision LLM
- `SentenceTransformersDocumentEmbedder`: Creates high-quality text embeddings
- `mixedbread-ai/mxbai-embed-large-v1`: A powerful text embedding model

In [ ]:
from haystack import Pipeline
from haystack.components.converters import PyPDFToDocument
from haystack.components.converters.image import ImageFileToDocument
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.components.extractors.image import LLMDocumentContentExtractor
from haystack.components.joiners import DocumentJoiner
from haystack.components.preprocessors.document_splitter import DocumentSplitter
from haystack.components.routers.file_type_router import FileTypeRouter
from haystack.components.writers.document_writer import DocumentWriter
from haystack.document_stores.in_memory import InMemoryDocumentStore

# Create new document store
doc_store_text = InMemoryDocumentStore(embedding_similarity_function="cosine")

# Define components
file_type_router = FileTypeRouter(mime_types=["application/pdf", "image/jpeg"])
image_converter = ImageFileToDocument()
pdf_converter = PyPDFToDocument()
pdf_splitter = DocumentSplitter(split_by="page", split_length=1)
final_doc_joiner = DocumentJoiner(sort_by_score=False)
document_writer = DocumentWriter(doc_store_text)

# Text-only embedder (better performance)
doc_embedder = SentenceTransformersDocumentEmbedder(
    model="mixedbread-ai/mxbai-embed-large-v1",
    progress_bar=False
)

llm_generator = OpenAIChatGenerator(model="gpt-4o-mini"),
# optional - if you want to use open source model instead of OpenAI
# from haystack_integrations.components.generators.ollama import OllamaGenerator, OllamaChatGenerator
# llm_generator = OllamaChatGenerator(model="mistral-nemo:12b",
#                             generation_kwargs={
#                               "num_predict": 100,
#                               "temperature": 0.9,
#                             })

# LLM content extractor for images
llm_content_extractor = LLMDocumentContentExtractor(
    chat_generator= llm_generator,
    max_workers=1
)



print("Components initialized!")

Components initialized!


## Create Custom ImagePathFixer Component

In [11]:
from haystack import component
from haystack.dataclasses import Document
from typing import List
import os

@component
class ImagePathFixer:
    """
    Fixes the file paths in image documents to be absolute paths.
    ImageFileToDocument only stores the basename, so we need to restore the full path.
    """
    
    @component.output_types(documents=List[Document])
    def run(self, documents: List[Document]) -> dict:
        """Fix the file paths in documents to be absolute."""
        for doc in documents:
            if "file_path" in doc.meta:
                file_path = doc.meta["file_path"]
                # If it's just a filename without directory, assume it's in the images folder
                if os.path.basename(file_path) == file_path:
                    # Get the notebook directory
                    notebook_dir = os.path.dirname(os.path.abspath("3_multimodal_indexing_llm.ipynb")) if os.path.exists("3_multimodal_indexing_llm.ipynb") else os.getcwd()
                    doc.meta["file_path"] = os.path.join(notebook_dir, "images", file_path)
        
        return {"documents": documents}

print("Custom ImagePathFixer component defined!")

Custom ImagePathFixer component defined!


## Build the Indexing Pipeline

Now we'll connect all components. The key difference from Approach 1 is the LLM content extractor.

**Pipeline Flow**:

**PDF Branch**:
1. File Router → PDF Converter
2. PDF Converter → PDF Splitter
3. PDF Splitter → Document Joiner

**Image Branch**:
1. File Router → Image Converter
2. Image Converter → Path Fixer
3. Path Fixer → LLM Content Extractor (Vision LLM)
4. LLM Content Extractor → Document Joiner

**Final Steps**:
1. Document Joiner → Text Embedder
2. Text Embedder → Document Writer → Store

In [12]:
# Build the indexing pipeline with LLM extraction
indexing_pipe_text = Pipeline()
indexing_pipe_text.add_component("file_type_router", file_type_router)
indexing_pipe_text.add_component("pdf_converter", pdf_converter)
indexing_pipe_text.add_component("pdf_splitter", pdf_splitter)
indexing_pipe_text.add_component("image_converter", image_converter)
indexing_pipe_text.add_component("image_path_fixer", ImagePathFixer())  
indexing_pipe_text.add_component("llm_content_extractor", llm_content_extractor)
indexing_pipe_text.add_component("doc_embedder", doc_embedder)
indexing_pipe_text.add_component("final_doc_joiner", final_doc_joiner)
indexing_pipe_text.add_component("document_writer", document_writer)

# Connect components
indexing_pipe_text.connect("file_type_router.application/pdf", "pdf_converter.sources")
indexing_pipe_text.connect("pdf_converter.documents", "pdf_splitter.documents")
indexing_pipe_text.connect("pdf_splitter.documents", "final_doc_joiner.documents")
indexing_pipe_text.connect("file_type_router.image/jpeg", "image_converter.sources")
indexing_pipe_text.connect("image_converter.documents", "image_path_fixer.documents") 
indexing_pipe_text.connect("image_path_fixer.documents", "llm_content_extractor.documents")  
indexing_pipe_text.connect("llm_content_extractor.documents", "final_doc_joiner.documents")
indexing_pipe_text.connect("final_doc_joiner.documents", "doc_embedder.documents")
indexing_pipe_text.connect("doc_embedder.documents", "document_writer.documents")

🚅 Components
  - file_type_router: FileTypeRouter
  - pdf_converter: PyPDFToDocument
  - pdf_splitter: DocumentSplitter
  - image_converter: ImageFileToDocument
  - image_path_fixer: ImagePathFixer
  - llm_content_extractor: LLMDocumentContentExtractor
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - final_doc_joiner: DocumentJoiner
  - document_writer: DocumentWriter
🛤️ Connections
  - file_type_router.application/pdf -> pdf_converter.sources (list[Union[str, Path, ByteStream]])
  - file_type_router.image/jpeg -> image_converter.sources (list[Union[str, Path, ByteStream]])
  - pdf_converter.documents -> pdf_splitter.documents (list[Document])
  - pdf_splitter.documents -> final_doc_joiner.documents (list[Document])
  - image_converter.documents -> image_path_fixer.documents (list[Document])
  - image_path_fixer.documents -> llm_content_extractor.documents (List[Document])
  - llm_content_extractor.documents -> final_doc_joiner.documents (list[Document])
  - doc_embedder.docu

## Visualize the Pipeline

In [13]:
# Visualize the pipeline
os.makedirs("images", exist_ok=True)
indexing_pipe_text.draw(path="images/multimodal_indexing_text.png")

![](./images/multimodal_indexing_text.png)

![](./images/multimodal_indexing_text.png)

## Run the Indexing Pipeline

In [14]:
# Run the indexing pipeline
indexing_result = indexing_pipe_text.run(
    data={"file_type_router": {"sources": [
        os.path.join(notebook_dir, "data_for_indexing/attention_is_all_you_need.pdf"),
        os.path.join(notebook_dir, "images/apple.jpg")
    ]}}
)

indexed_documents = doc_store_text.filter_documents()
print(f"Indexed {len(indexed_documents)} documents")

LLM tuple execution failed. Skipping metadata extraction. Failed with exception ''tuple' object has no attribute 'run''.


Indexed 15 documents


## Inspect the Extracted Content

Let's see what the vision LLM extracted from our image:

In [15]:
# Inspect the extracted content from the image
image_docs = [d for d in indexed_documents if "apple.jpg" in d.meta.get("file_path", "")]
if image_docs:
    print("Extracted image content:")
    print(image_docs[0].content)
else:
    print("No image documents found")

No image documents found


## Compare with Approach 1

Let's check a few documents to see the difference:

In [16]:
# Check the first few documents
for i, doc in enumerate(indexed_documents[:3]):
    print(f"\nDocument {i+1}:")
    print(f"Type: {doc.meta.get('file_path', 'Unknown')}")
    print(f"Has embedding: {doc.embedding is not None}")
    if doc.embedding:
        print(f"Embedding dimension: {len(doc.embedding)}")
    print(f"Content preview: {str(doc.content)[:150]}...")


Document 1:
Type: attention_is_all_you_need.pdf
Has embedding: True
Embedding dimension: 1024
Content preview: Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalis...

Document 2:
Type: attention_is_all_you_need.pdf
Has embedding: True
Embedding dimension: 1024
Content preview: 1 Introduction
Recurrent neural networks, long short-term memory [13] and gated recurrent [7] neural networks
in particular, have been firmly establis...

Document 3:
Type: attention_is_all_you_need.pdf
Has embedding: True
Embedding dimension: 1024
Content preview: Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
c...


## Summary

In this notebook, we:

1. **Built a multimodal indexing pipeline** using LLM content extraction
2. **Extracted textual descriptions** from images using GPT-4o-mini
3. **Created text embeddings** for both PDFs and image descriptions
4. **Stored everything** in a unified text embedding space

### Key Advantages of this Approach:
- **Richer semantic understanding**: The LLM can describe images in detail
- **Better for text queries**: Works well with natural language questions
- **Unified embedding space**: Everything is embedded as text

### Trade-offs:
- **Slower**: Requires LLM API calls
- **More expensive**: OpenAI API costs
- **Information loss**: Visual details may be lost in text description

### When to Use This Approach:
- Question answering about images
- Semantic search across documents
- When text understanding is more important than visual matching

### Next Steps:
- Continue to notebook 4 to build a complete RAG pipeline using this indexed data
- Compare retrieval quality with Approach 1
- Experiment with different vision LLMs